# 7. Automation with Python

## OAuth 2

In [1]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import os.path

In [3]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

In [4]:
flow = InstalledAppFlow.from_client_secrets_file('client_credentials.json', SCOPES)

In [5]:
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
else:
    creds = flow.run_local_server()
    with open('token.json', 'w') as f:
        f.write(creds.to_json())

In [6]:
service = build('gmail', 'v1', credentials=creds)

## Connecting to the Gmail API

In [7]:
msgs = service.users().messages().list(userId='me').execute()

In [8]:
msgs.keys()

dict_keys(['messages', 'nextPageToken', 'resultSizeEstimate'])

In [9]:
msgs['messages']

[{'id': '183c18b1212a45ad', 'threadId': '183c18b1212a45ad'},
 {'id': '183c18a29e44d276', 'threadId': '183c18a29e44d276'},
 {'id': '183c06e0c41a7e88', 'threadId': '183c06e0c41a7e88'},
 {'id': '183c06d211213c80', 'threadId': '183c06d211213c80'},
 {'id': '183c06c356dd2070', 'threadId': '183c06c356dd2070'},
 {'id': '183c06b4af94933f', 'threadId': '183c06b4af94933f'},
 {'id': '183c06a5d89586c1', 'threadId': '183c06a5d89586c1'},
 {'id': '183c069700c2e8c1', 'threadId': '183c069700c2e8c1'},
 {'id': '183c06884f766492', 'threadId': '183c06884f766492'},
 {'id': '183c067997039a8b', 'threadId': '183c067997039a8b'},
 {'id': '183c066ac34bf28a', 'threadId': '183c066ac34bf28a'},
 {'id': '183c065c3ebd7682', 'threadId': '183c065c3ebd7682'},
 {'id': '183c064d4ae0ae4c', 'threadId': '183c064d4ae0ae4c'},
 {'id': '183c063eb07aa557', 'threadId': '183c063eb07aa557'},
 {'id': '183c062ffaabc057', 'threadId': '183c062ffaabc057'},
 {'id': '183c062124248e5e', 'threadId': '183c062124248e5e'},
 {'id': '183c061271eac5e

In [10]:
msgs = service.users().messages().list(userId = 'me', pageToken=msgs['nextPageToken']).execute()

In [11]:
msgs = service.users().messages().list(userId='me').execute()
message_list = msgs['messages']

while 'nextPageToken' in msgs:
    msgs = (service.users()
                   .messages()
                   .list(userId='me',
                         pageToken = msgs['nextPageToken'])
                   .execute())
    message_list.extend(msgs['messages'])

In [12]:
len(message_list)

1385

In [13]:
msgs = service.users().messages().list(userId='me', q='subject:("Bail Paid")').execute()

In [14]:
msgs

{'messages': [{'id': '183c0603b81dabdf', 'threadId': '183c0603b81dabdf'},
  {'id': '183c0517a9af29e8', 'threadId': '183c0517a9af29e8'},
  {'id': '183c04bf59528752', 'threadId': '183c04bf59528752'},
  {'id': '183c043a8f93bc67', 'threadId': '183c043a8f93bc67'},
  {'id': '183c03976100ccb4', 'threadId': '183c03976100ccb4'},
  {'id': '183c02709bd0e215', 'threadId': '183c02709bd0e215'},
  {'id': '183c0226f616ada1', 'threadId': '183c0226f616ada1'},
  {'id': '183c017609e7fd6b', 'threadId': '183c017609e7fd6b'},
  {'id': '183c0149a3940eff', 'threadId': '183c0149a3940eff'},
  {'id': '183bff372e36f3f2', 'threadId': '183bff372e36f3f2'},
  {'id': '183bfe3caf5a5d3d', 'threadId': '183bfe3caf5a5d3d'},
  {'id': '183bfd3392b45aa4', 'threadId': '183bfd3392b45aa4'},
  {'id': '183bfc47caea257e', 'threadId': '183bfc47caea257e'},
  {'id': '183bfbe0a367486c', 'threadId': '183bfbe0a367486c'},
  {'id': '183bf61c3774a968', 'threadId': '183bf61c3774a968'},
  {'id': '183bd342eaf49ecf', 'threadId': '183bd342eaf49ecf

In [15]:
message_list[-1]

{'id': '183b9094e2f125e3', 'threadId': '183b9094e2f125e3'}

In [16]:
msg = service.users().messages().get(userId='me', 
                                     id=message_list[-1]['id']).execute()

In [17]:
msg.keys()

dict_keys(['id', 'threadId', 'labelIds', 'snippet', 'payload', 'sizeEstimate', 'historyId', 'internalDate'])

In [18]:
msg['payload']

{'partId': '',
 'mimeType': 'multipart/alternative',
 'filename': '',
 'headers': [{'name': 'Delivered-To',
   'value': 'cbs.python.bailfund@gmail.com'},
  {'name': 'Received',
   'value': 'by 2002:a05:6359:4e09:b0:c8:3b63:18e9 with SMTP id oq9csp1306737rwb;        Sat, 8 Oct 2022 12:18:58 -0700 (PDT)'},
  {'name': 'X-Received',
   'value': 'by 2002:a63:1c4e:0:b0:458:e183:1342 with SMTP id c14-20020a631c4e000000b00458e1831342mr9808299pgm.409.1665256738674;        Sat, 08 Oct 2022 12:18:58 -0700 (PDT)'},
  {'name': 'ARC-Seal',
   'value': 'i=1; a=rsa-sha256; t=1665256738; cv=none;        d=google.com; s=arc-20160816;        b=HZzzKUe0MyjXuZe/UaLoukmXC8H7YNo5x0544qi3trwescmHy5G/UYMP2jCuCWhhrv         IQ1uZ9b6lE3e6aq98wbu0R2CYAGxzQgG7muic67/UZI/xmCQi2H5y8bjhAHJmHgO2Cj4         jXCQ324C2L+CH38I5zXwTl+cG6cZklXN3goft3STxUAhNpczTjt7dl3TFri/O1SQnQgm         nUsjMB7zbLDmZkhy5w05uWuohSbr8QLRxVZ74Ctn6IYmyVAIxIM6FAtdZFlNWghG2E/x         SJYbnLRkwBSpR5LPuAzkGjYaRrWxgRua0yGMRNYdzZH2IMTa0B/6PA1zjw/Se

In [19]:
from pprint import pprint

In [20]:
pprint(msg['payload'])

{'body': {'size': 0},
 'filename': '',
 'headers': [{'name': 'Delivered-To', 'value': 'cbs.python.bailfund@gmail.com'},
             {'name': 'Received',
              'value': 'by 2002:a05:6359:4e09:b0:c8:3b63:18e9 with SMTP id '
                       'oq9csp1306737rwb;        Sat, 8 Oct 2022 12:18:58 '
                       '-0700 (PDT)'},
             {'name': 'X-Received',
              'value': 'by 2002:a63:1c4e:0:b0:458:e183:1342 with SMTP id '
                       'c14-20020a631c4e000000b00458e1831342mr9808299pgm.409.1665256738674;        '
                       'Sat, 08 Oct 2022 12:18:58 -0700 (PDT)'},
             {'name': 'ARC-Seal',
              'value': 'i=1; a=rsa-sha256; t=1665256738; cv=none;        '
                       'd=google.com; s=arc-20160816;        '
                       'b=HZzzKUe0MyjXuZe/UaLoukmXC8H7YNo5x0544qi3trwescmHy5G/UYMP2jCuCWhhrv         '
                       'IQ1uZ9b6lE3e6aq98wbu0R2CYAGxzQgG7muic67/UZI/xmCQi2H5y8bjhAHJmHgO2Cj4         '

In [21]:
msg['payload']['parts'][0]['body']['data']

'U2VudCBEYXRlID0gMDktMDEtMjAyMSAxMTo1NDoyMA0KDQpDYXNlIE51bWJlciA9IENSLTQ3MjUzNzUNCkRlZmVuZGFudCA9IEtyaXN0YSBSaWdncw0KRE9CID0gMTItMjUtMTk4OQ0KU2V4ID0gRg0KQ2hhcmdlID0gUG9zc2Vzc2lvbiBvZiBlYXZlc2Ryb3BwaW5nIGRldmljZXMtIENsYXNzIEEgTWlzZGVtZWFub3INCkFycmVzdCBEYXRlID0gMDktMDEtMjAyMSAxMTozMjowMA0KDQpVcGRhdGUgZGV0YWlscw0KDQpQdXJwb3NlID0gQ2FzZSBvcGVuZWQNCkxvY2F0aW9uID0gU3ByaW5nZmllbGQNCk1vZGFsaXR5ID0gVmlydHVhbA0KDQpDYXNlIENsb3NlZCA9IE5PDQoNCi0tDQpDQlMgUHl0aG9uIEJhaWwgRnVuZCBDYXNlDQo='

In [22]:
import base64

In [23]:
print(base64.b64decode(msg['payload']['parts'][0]['body']['data'].replace('-', '+').replace('_', '/')).decode('utf-8'))

Sent Date = 09-01-2021 11:54:20

Case Number = CR-4725375
Defendant = Krista Riggs
DOB = 12-25-1989
Sex = F
Charge = Possession of eavesdropping devices- Class A Misdemeanor
Arrest Date = 09-01-2021 11:32:00

Update details

Purpose = Case opened
Location = Springfield
Modality = Virtual

Case Closed = NO

--
CBS Python Bail Fund Case



## Creating PowerPoint Files in Python

In [24]:
!pip install python-pptx

In [25]:
import pptx

In [26]:
pres = pptx.Presentation()

In [27]:
pres.slides

In [28]:
len(pres.slides)

0

In [29]:
pres.save('file_name.pptx')

### Adding Slides

In [30]:
title_layout = pres.slide_layouts[0]

In [31]:
slide = pres.slides.add_slide(title_layout)

In [32]:
len(pres.slides)

1

In [33]:
blank_slide = pres.slides.add_slide(pres.slide_layouts[6])

### Adding and Editing Content on Slides

In [34]:
len(slide.shapes)

2

In [35]:
len(blank_slide.shapes)

0

In [36]:
len(pres.slides[0].shapes)

2

In [37]:
pres.slides[0].shapes[0].text = 'Hello World!!' 
pres.slides[0].shapes[1].text = 'Why I love Python'

In [38]:
pres.save('file_name.pptx')

In [39]:
shape = (pres.slides[1]
             .shapes
             .add_textbox(left = pptx.util.Inches(0.5), 
                          top = pptx.util.Inches(0.5),
                          width = pptx.util.Inches(8), 
                          height = pptx.util.Inches(0.5)))

shape.text = 'This is a table'

In [40]:
shape = (pres.slides[1]
             .shapes
             .add_table(rows = 3, 
                        cols = 4,
                        left = pptx.util.Inches(0.5), 
                        top = pptx.util.Inches(1.5), 
                        width = pptx.util.Inches(8), 
                        height = pptx.util.Inches(2)))

shape.table.cell(2,2).text = 'Sample content' 

for i in range(4):
    shape.table.cell(0, i).text = f'Header {i+1}'

In [41]:
pres.save('file_name.pptx')

In [42]:
import pptx.chart.data

In [43]:
slide = pres.slides.add_slide(pres.slide_layouts[6])

In [44]:
chart_data = pptx.chart.data.ChartData()

In [45]:
chart_data.categories = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun'] 
chart_data.add_series('Series 1', (1, 2, 5, 8, 3, 5))

In [46]:
chart = slide.shapes.add_chart(chart_type=pptx.enum.chart.XL_CHART_TYPE.LINE,
                               x=pptx.util.Inches(2), 
                               y=pptx.util.Inches(2), 
                               cx=pptx.util.Inches(6), 
                               cy=pptx.util.Inches(4.5), 
                               chart_data=chart_data).chart

In [47]:
chart.has_legend=False

In [48]:
pres.save('file_name.pptx')